3 - Seja o conjunto de dados disponibilizado na pasta Questão 3 (imagens e anotações em arquivo .json associado à construções, sobre imagens satelitais), implemente algum modelo responsável pela segmentação das residências presentes as imagens satelitais disponibilizadas.

In [12]:
import os
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU, Precision, Recall
from tensorflow.keras.optimizers import Adam

In [16]:
# Função para carregar os dados e as anotações
def load_data(image_folder, annotation_file_1, annotation_file_2):
    images = []
    annotations_1 = []
    annotations_2 = []

    # Carregar imagens da pasta
    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        if img_path.endswith(".jpg"):
            img = Image.open(img_path)
            images.append(img)

    # Carregar anotações dos arquivos
    with open(annotation_file_1, 'r') as file:
        data_1 = json.load(file)
        annotations_1 = data_1["annotations"]

    with open(annotation_file_2, 'r') as file:
        data_2 = json.load(file)
        annotations_2 = data_2["annotations"]

    # Combinar as anotações
    combined_annotations = []
    for annot_1, annot_2 in zip(annotations_1, annotations_2):
        combined_annotation = np.logical_or(annot_1, annot_2)
        combined_annotations.append(combined_annotation)

    return np.array(images), np.array(combined_annotations)

# Carregar dados e anotações
image_folder = "../ex3/building/train/images/"
annotation_file_1 = "../ex3/building/annotation-small.json"
annotation_file_2 = "../ex3/building/annotation.json"
X, y = load_data(image_folder, annotation_file_1, annotation_file_2)


In [9]:
# Dividir dados em conjuntos de treinamento e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir a arquitetura da rede neural
def create_model(activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, padding='same', input_shape=(img_height, img_width, 3)),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(64, (3, 3), activation=activation, padding='same'),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(128, (3, 3), activation=activation, padding='same'),
        MaxPooling2D((2, 2), padding='same'),
        Conv2D(256, (3, 3), activation=activation, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation, padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])
    model.compile(optimizer=optimizer, loss=BinaryCrossentropy(), metrics=[MeanIoU(num_classes=2), Precision(), Recall()])
    return model

In [5]:
# Definir hiperparâmetros para grid search
param_grid = {
    'activation': ['relu', 'elu'],
    'optimizer': ['adam', 'rmsprop']
}

# Criar o modelo Keras para grid search
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)

# Realizar busca em grade
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid_search.fit(X_train, y_train)

# Melhores parâmetros encontrados
best_params = grid_result.best_params_
print("Melhores Parâmetros:", best_params)

In [6]:
# Treinar o modelo com os melhores parâmetros
best_model = create_model(activation=best_params['activation'], optimizer=best_params['optimizer'])
history = best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Avaliar o modelo com métricas diferentes
loss, mean_iou, precision, recall = best_model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Mean IoU:", mean_iou)
print("Precision:", precision)
print("Recall:", recall)

FileNotFoundError: [Errno 2] No such file or directory: 'data.json'